<a href="https://colab.research.google.com/github/sugarforever/Advanced-RAG/blob/main/01_semi_structured_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advanced RAG - 01. RAG on Semi-structured data

**What is RAG?**

Retrieval augmented generation (RAG) is a natural language processing (NLP) technique that employes the capabilities of retrieval and generative based AI models.

**What is Naive RAG?**

Naive RAG often refers to splitting documents into chunks, embedding them, and retrieving chunks based on semantic similarity search to a user question.

It's simple, but of poor overall performance.

**That's why we need Advanced RAG.**

In this tutorials (**Advanced RAG**), we will learn the techniques and best practices in RAG application development, that can improve the quality of the RAG.

It's crucial to the success of a RAG application.

## 01. RAG on Semi-structured data

### Introduction

#### ✏️ What is Structured Data?

Structured data is organized information with a predefined format, typically stored in rows and columns, making it easy to search and analyze.

#### ✏️ What is Unstructured Data?

Unstructured data is information that lacks a specific format or organization, often in the form of text, images, or multimedia, making it challenging to analyze without specialized techniques.

#### ✏️ What is Semi-structured Data?

Apparently, semi-structured data is the mix of them above.

It's challenging for RAG to process semi-structured data, as:

1. Text splitting may break up tables
2. Tables and images are challenging for embedding and semantic search

The typical semi-structured data is PDF document that contains text, tables, images and so on.

In this tutorial, let's use the following components to showcase how to build RAG on top of semi-structured data:

1. ✂️ [unstructured](https://github.com/Unstructured-IO/unstructured)
  
  Open source libraries and APIs to build custom preprocessing pipelines for labeling, training, or production machine learning pipelines.

  We will use it to parse PDF documents and extract different types of elements seperately, such as text, table, and image

2. 🦜 [LangChain](https://github.com/langchain-ai/langchain)

3. 🗂 [Chromadb](https://github.com/chroma-core/chroma)

  Vector data storage

The PDF document we use in this example is the [NVIDIA Statement of Changes](https://d18rn0p25nwr6d.cloudfront.net/CIK-0001045810/381953f9-934e-4cc8-b099-144910676bad.pdf). It's a small PDF file containing several tables which is a good example for quick data processing and clear demonstration.

### Prepare Environment

Let's install the necessary Python packages.

In [18]:
!pip install langchain unstructured[all-docs] pydantic lxml openai chromadb tiktoken -q -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 8.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.


Download the PDF file and name it as `statement_of_changes.pdf`.

In [1]:
!wget -O statement_of_changes.pdf https://d18rn0p25nwr6d.cloudfront.net/CIK-0001045810/381953f9-934e-4cc8-b099-144910676bad.pdf

--2024-04-07 09:21:25--  https://d18rn0p25nwr6d.cloudfront.net/CIK-0001045810/381953f9-934e-4cc8-b099-144910676bad.pdf
Resolving d18rn0p25nwr6d.cloudfront.net (d18rn0p25nwr6d.cloudfront.net)... 13.35.21.55, 13.35.21.95, 13.35.21.156, ...
Connecting to d18rn0p25nwr6d.cloudfront.net (d18rn0p25nwr6d.cloudfront.net)|13.35.21.55|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 119037 (116K) [application/pdf]
Saving to: ‘statement_of_changes.pdf’

statement_of_change 100%[===================>] 116.25K   686KB/s    in 0.2s    

2024-04-07 09:21:26 (686 KB/s) - ‘statement_of_changes.pdf’ saved [119037/119037]



Install required platform packages:

- poppler-utils
  
  A collection of command-line utilities built on Poppler's library API, to manage PDF and extract contents

- tesseract-ocr

  Optical character recognition engine

In [2]:
!apt-get install poppler-utils tesseract-ocr

/bin/bash: apt-get: command not found


In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "Your Valid OpenAI API Key"

### Coding

1. Use `unstructured` library to partition the PDF document into different type of elements.

In [4]:
from typing import Any

from pydantic import BaseModel
from unstructured.partition.pdf import partition_pdf

raw_pdf_elements = partition_pdf(
    filename="statement_of_changes.pdf",
    extract_images_in_pdf=False,
    infer_table_structure=True,
    chunking_strategy="by_title",
    max_characters=4000,
    new_after_n_chars=3800,
    combine_text_under_n_chars=2000,
    image_output_dir_path=".",
)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


yolox_l0.05.onnx:   0%|          | 0.00/217M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/115M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/46.8M [00:00<?, ?B/s]

Some weights of the model checkpoint at microsoft/table-transformer-structure-recognition were not used when initializing TableTransformerForObjectDetection: ['model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


2. Categorize the elements

In [30]:
category_counts = {}

for element in raw_pdf_elements:
    category = str(type(element))
    if category in category_counts:
        category_counts[category] += 1
    else:
        category_counts[category] = 1

unique_categories = set(category_counts.keys())
category_counts

{"<class 'unstructured.documents.elements.CompositeElement'>": 5,
 "<class 'unstructured.documents.elements.Table'>": 4}

In [31]:
class Element(BaseModel):
    type: str
    text: Any

table_elements = []
text_elements = []
for element in raw_pdf_elements:
    if "unstructured.documents.elements.Table" in str(type(element)):
        table_elements.append(Element(type="table", text=str(element)))
    elif "unstructured.documents.elements.CompositeElement" in str(type(element)):
        text_elements.append(Element(type="text", text=str(element)))

In [32]:
print(len(table_elements))
print(len(text_elements))

4
5


In [33]:
table_elements[0]

Element(type='table', text='1. Name and Address of Reporting Person = \\Drell Persis 2. Issuer Name and Ticker or Trading Symbol INVIDIA CORP [ NVDA ] 5. Relationship of Reporting Person(s) to Issuer (Check all applicable) 3. Date of Earliest Transaction (MM/DD/YYYY) |_X_Director 10% Owner (es) (Fis) (Middle) Officer (give title below) Other (specify below) C/O NVIDIA CORPORATION, 2788 10/6/2023 SAN TOMAS EXPRESSWAY (Street) 4. If Amendment, Date Original Filed (MM/DD/YYYY) 6. Individual or Joint/Group Filing (Check Applicable Line) SANTA CLARA, CA 95051 | X_ Form filed by One Reporting Person - |__ Form filed by More than One Reporting Person (City) (State) (Zip)')

In [34]:
table_elements[2]

Element(type='table', text='1. Title of Derivate |2. 3. Trans. 3A. Deemed |4. Trans. Code |5. Number of 6. Date Exercisable 7. Title and Amount of 8. Price of ]9. Number of | 10. Security Conversion | Date Execution |(Instr, 8) Derivative Securities |and Expiration Date _| Securities Underlying Derivative }derivative | Ownership] (Instr. 3) or Exercise Date, if any Acquired (A) or Derivative Security Security |Securities |Formof Price of Disposed of (D) (Instr. 3 and 4) (Instr. 5) |Beneficially | Derivative | Derivative (Instr. 3, 4 and 5) Owned Security: | Security Following —_| Direct (D) - Reported _| or Indirect Date Expiration] 1... | Amount or Number of Transaction(s)] (1) (Instr. coe |v | (a) (D) | Exercisable|Date Shares (instr. 4) 4)')

In [35]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser

3. Build up summarization chain with LangChain framework

In [36]:
prompt_text = """
  You are responsible for concisely summarizing table or text chunk:

  {element}
"""
prompt = ChatPromptTemplate.from_template(prompt_text)
summarize_chain = {"element": lambda x: x} | prompt | ChatOpenAI(temperature=0, model="gpt-3.5-turbo") | StrOutputParser()

4. Summarize each text and table element

In [12]:
tables = [i.text for i in table_elements]
table_summaries = summarize_chain.batch(tables, {"max_concurrency": 5})

texts = [i.text for i in text_elements]
text_summaries = summarize_chain.batch(texts, {"max_concurrency": 5})

5. Use LangChain MultiVectorRetriever to associate summaries of tables and texts with original text chunks in parent-child relationship.

In [19]:
import uuid

from langchain.embeddings import OpenAIEmbeddings
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.schema.document import Document
from langchain.storage import InMemoryStore
from langchain.vectorstores import Chroma

id_key = "doc_id"

# The retriever (empty to start)
retriever = MultiVectorRetriever(
    vectorstore=Chroma(collection_name="summaries", embedding_function=OpenAIEmbeddings()),
    docstore=InMemoryStore(),
    id_key=id_key,
)

# Add texts
doc_ids = [str(uuid.uuid4()) for _ in texts]
summary_texts = [
    Document(page_content=s, metadata={id_key: doc_ids[i]})
    for i, s in enumerate(text_summaries)
]
retriever.vectorstore.add_documents(summary_texts)
retriever.docstore.mset(list(zip(doc_ids, texts)))

# Add tables
table_ids = [str(uuid.uuid4()) for _ in tables]
summary_tables = [
    Document(page_content=s, metadata={id_key: table_ids[i]})
    for i, s in enumerate(table_summaries)
]
retriever.vectorstore.add_documents(summary_tables)
retriever.docstore.mset(list(zip(table_ids, tables)))

In [37]:
from langchain.schema.runnable import RunnablePassthrough

template = """Answer the question based only on the following context, which can include text and tables:
{context}
Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

# RAG pipeline
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | ChatOpenAI(temperature=0, model="gpt-4")
    | StrOutputParser()
)

In [38]:
chain.invoke("How many stocks were disposed? Who is the beneficial owner?")

'2300 stocks were disposed. The beneficial owner is the Welch-Drell 2009 Revocable Trust.'

6. Experiment with GPT-3.5

Looks it doesn't perform as well as GPT-4.

In [39]:
# RAG pipeline
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | ChatOpenAI(temperature=0, model="gpt-3.5-turbo")
    | StrOutputParser()
)
chain.invoke("How many stocks were disposed? Who is the beneficial owner?")

'Based on the given context, it is not possible to determine how many stocks were disposed or who the beneficial owner is. The context does not provide any specific information about the disposal of stocks or the identification of the beneficial owner.'